<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#データ準備" data-toc-modified-id="データ準備-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>データ準備</a></span></li><li><span><a href="#データ読み込み" data-toc-modified-id="データ読み込み-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>データ読み込み</a></span></li><li><span><a href="#がっちゃんこ" data-toc-modified-id="がっちゃんこ-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>がっちゃんこ</a></span><ul class="toc-item"><li><span><a href="#レコード数を1年分（35,040件）に戻す" data-toc-modified-id="レコード数を1年分（35,040件）に戻す-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>レコード数を1年分（35,040件）に戻す</a></span></li></ul></li><li><span><a href="#教師データと訓練データの作成の作成" data-toc-modified-id="教師データと訓練データの作成の作成-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>教師データと訓練データの作成の作成</a></span><ul class="toc-item"><li><span><a href="#訓練データ" data-toc-modified-id="訓練データ-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>訓練データ</a></span></li><li><span><a href="#教師データ" data-toc-modified-id="教師データ-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>教師データ</a></span></li></ul></li><li><span><a href="#Chainerで予測" data-toc-modified-id="Chainerで予測-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Chainerで予測</a></span><ul class="toc-item"><li><span><a href="#標準化" data-toc-modified-id="標準化-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>標準化</a></span></li><li><span><a href="#TupleDatasetを作成" data-toc-modified-id="TupleDatasetを作成-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>TupleDatasetを作成</a></span></li><li><span><a href="#Iteratorの作成" data-toc-modified-id="Iteratorの作成-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Iteratorの作成</a></span></li><li><span><a href="#ライブラリーのインポート" data-toc-modified-id="ライブラリーのインポート-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>ライブラリーのインポート</a></span></li><li><span><a href="#LSTMネットワークの作成" data-toc-modified-id="LSTMネットワークの作成-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>LSTMネットワークの作成</a></span></li><li><span><a href="#モデルの作成" data-toc-modified-id="モデルの作成-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>モデルの作成</a></span></li><li><span><a href="#オプティマイザの設定" data-toc-modified-id="オプティマイザの設定-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>オプティマイザの設定</a></span></li><li><span><a href="#Updaterの作成" data-toc-modified-id="Updaterの作成-5.8"><span class="toc-item-num">5.8&nbsp;&nbsp;</span>Updaterの作成</a></span></li><li><span><a href="#Trainerの準備" data-toc-modified-id="Trainerの準備-5.9"><span class="toc-item-num">5.9&nbsp;&nbsp;</span>Trainerの準備</a></span></li><li><span><a href="#実行" data-toc-modified-id="実行-5.10"><span class="toc-item-num">5.10&nbsp;&nbsp;</span>実行</a></span></li></ul></li><li><span><a href="#モデルの保存" data-toc-modified-id="モデルの保存-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>モデルの保存</a></span></li><li><span><a href="#モデルの読み込み" data-toc-modified-id="モデルの読み込み-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>モデルの読み込み</a></span></li><li><span><a href="#学習、評価にも使用していないデータで検証" data-toc-modified-id="学習、評価にも使用していないデータで検証-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>学習、評価にも使用していないデータで検証</a></span></li></ul></div>

## データ準備

In [1]:
import pandas as pd 
import numpy as np
import copy
from datetime import datetime as dt

from matplotlib import pyplot as plt
import seaborn as sns

# warningの抑止
import warnings
warnings.filterwarnings('ignore')

# メモリ解放
import gc

## データ読み込み

In [ ]:
pd.set_option("display.max_columns", 80)

# データを読み込む
group_a_df = pd.read_csv("data/train_final_group_a.csv")
group_b_df = pd.read_csv("data/train_final_group_b.csv")
group_c_df = pd.read_csv("data/train_final_group_c.csv")

group_a_df.shape, group_b_df.shape, group_c_df.shape

## がっちゃんこ

In [ ]:
df = pd.concat([group_a_df, group_b_df,group_c_df])
display(df.head(), df.shape)

### レコード数を1年分（35,040件）に戻す

In [ ]:
ts_group = df.groupby(["year", "month", "day", "hour", "minute"])
ts_df = ts_group.count()

display(ts_df.head(), ts_df.shape)

In [ ]:
# 複合インデックスを解除
ts_df.reset_index(inplace=True)

display(ts_df.head(), ts_df.shape)

In [ ]:
ts_df["area_a"] = df.query("CALL_TYPE == 'A'")["area_a"]
ts_df["area_b"] = df.query("CALL_TYPE == 'B'")["area_b"]
ts_df["area_c_other"] = df.query("CALL_TYPE == 'C'")["area_c_other"]
ts_df["area_c_00"] = df.query("CALL_TYPE == 'C'")["area_c_00"]
ts_df["area_c_01"] = df.query("CALL_TYPE == 'C'")["area_c_01"]
ts_df["area_c_02"] = df.query("CALL_TYPE == 'C'")["area_c_02"]
ts_df["area_c_03"] = df.query("CALL_TYPE == 'C'")["area_c_03"]
ts_df["area_c_04"] = df.query("CALL_TYPE == 'C'")["area_c_04"]
ts_df["area_c_10"] = df.query("CALL_TYPE == 'C'")["area_c_10"]
ts_df["area_c_11"] = df.query("CALL_TYPE == 'C'")["area_c_11"]
ts_df["area_c_12"] = df.query("CALL_TYPE == 'C'")["area_c_12"]
ts_df["area_c_13"] = df.query("CALL_TYPE == 'C'")["area_c_13"]
ts_df["area_c_14"] = df.query("CALL_TYPE == 'C'")["area_c_14"]
ts_df["area_c_20"] = df.query("CALL_TYPE == 'C'")["area_c_20"]
ts_df["area_c_21"] = df.query("CALL_TYPE == 'C'")["area_c_21"]
ts_df["area_c_22"] = df.query("CALL_TYPE == 'C'")["area_c_22"]
ts_df["area_c_23"] = df.query("CALL_TYPE == 'C'")["area_c_23"]
ts_df["area_c_24"] = df.query("CALL_TYPE == 'C'")["area_c_24"]
ts_df["area_c_30"] = df.query("CALL_TYPE == 'C'")["area_c_30"]
ts_df["area_c_31"] = df.query("CALL_TYPE == 'C'")["area_c_31"]
ts_df["area_c_32"] = df.query("CALL_TYPE == 'C'")["area_c_32"]
ts_df["area_c_33"] = df.query("CALL_TYPE == 'C'")["area_c_33"]
ts_df["area_c_34"] = df.query("CALL_TYPE == 'C'")["area_c_34"]
ts_df["area_c_40"] = df.query("CALL_TYPE == 'C'")["area_c_40"]
ts_df["area_c_41"] = df.query("CALL_TYPE == 'C'")["area_c_41"]
ts_df["area_c_42"] = df.query("CALL_TYPE == 'C'")["area_c_42"]
ts_df["area_c_43"] = df.query("CALL_TYPE == 'C'")["area_c_43"]
ts_df["area_c_44"] = df.query("CALL_TYPE == 'C'")["area_c_44"]

ts_df["a_dir"] = df.query("CALL_TYPE == 'A'")["direction"]
ts_df["b_dir"] = df.query("CALL_TYPE == 'B'")["direction"]
ts_df["c_dir"] = df.query("CALL_TYPE == 'C'")["direction"]

ts_df["a_rid"] = df.query("CALL_TYPE == 'A'")["riding_time_mean"]
ts_df["b_rid"] = df.query("CALL_TYPE == 'B'")["riding_time_mean"]
ts_df["c_rid"] = df.query("CALL_TYPE == 'C'")["riding_time_mean"]

ts_df["a_dis"] = df.query("CALL_TYPE == 'A'")["distance_mean"]
ts_df["b_dis"] = df.query("CALL_TYPE == 'B'")["distance_mean"]
ts_df["c_dis"] = df.query("CALL_TYPE == 'C'")["distance_mean"]

display(ts_df.head(), ts_df.shape)

In [ ]:
# 念の為、ソート
ts_df.sort_values(
    by=["year", "month", "day", "hour", "minute"],
    ascending=True,
    inplace=True)


# 要らない列を削除
ts_df.drop(["CALL_TYPE", "direction", "riding_time_mean", "distance_mean"],
        axis=1,
        inplace=True)

display(ts_df.head(), ts_df.shape)

## 教師データと訓練データの作成の作成

### 訓練データ

In [ ]:
28*672

周期性を考えて、1週間分の過去データ= 96*7 = 672 レコードを 入力とする。（でかいので４日分に変更）
1行、28*384 = 18816列

In [ ]:
d = pd.DataFrame()

a = pd.Series()
b = pd.Series([5, 6, 7])
c = pd.Series([10, 20, 40])

a = pd.concat([a, b], ignore_index=True)
a = pd.concat([a, c], ignore_index=True)

d = d.append(a, ignore_index = True)
d


In [ ]:
past_df = pd.DataFrame()

target_col = [
    'area_a', 'area_b', 'area_c_other', 'area_c_00', 'area_c_01', 'area_c_02',
    'area_c_03', 'area_c_04', 'area_c_10', 'area_c_11', 'area_c_12',
    'area_c_13', 'area_c_14', 'area_c_20', 'area_c_21', 'area_c_22',
    'area_c_23', 'area_c_24', 'area_c_30', 'area_c_31', 'area_c_32',
    'area_c_33', 'area_c_34', 'area_c_40', 'area_c_41', 'area_c_42',
    'area_c_43', 'area_c_44'
]

temp_series = pd.Series()
for i in range(384, 35040):

    if i % 100 == 0:
        print("### ", i)

    for j in range(i - 384, i):

        # 384レコード分、シリーズとして結合していく
        s = ts_df.iloc[j][target_col]
        temp_series = pd.concat([temp_series, s], ignore_index=True)

    # 過去の384レコード(=10752列)を行として追加
    past_df = past_df.append(temp_series, ignore_index=True)
    temp_series = pd.Series()

In [ ]:
past.to_csv("data/input_lstm.csv", index=False, header=True)
display(past.head(), past.shape)

### 教師データ

In [ ]:
ts_df.head()

In [ ]:
current_df = ts_df.drop(
    [
        "index", "year", "month", "day", "hour", "minute", "weekday",
        "holiday", "a_dir", "b_dir", "c_dir", "a_rid", "b_rid", "c_rid",
        "a_dis", "b_dis", "c_dis"
    ],
    axis=1,
)

current_df.to_csv("data/output_lstm.csv", index=False, header=True)
display(true_df.head(), true_df.shape)

## Chainerで予測

### 標準化

In [2]:
pd.set_option("display.max_columns", 80)

# データを読み込む
# 全体の35040中、[24:34680] のデータ
past_df = pd.read_csv("data/input_lstm_6hours.csv")
# past_df = pd.read_csv("data/input_lstm.csv")
# past_df = pd.read_csv("data/input_lstm_oneday.csv")

current_df = pd.read_csv("data/output_lstm_6hours.csv")

In [ ]:
past_df.shape, current_df.shape

In [3]:
from sklearn.preprocessing import StandardScaler

# 標準正規分布で標準化
scaler = StandardScaler()

X = np.array(past_df)
X = scaler.fit_transform(X)

Y = np.array(current_df)

In [4]:
del past_df
del current_df
gc.collect()

7

In [88]:
from sklearn.model_selection import train_test_split

# クロスバリデーション用にテストデータを分割(学習：75% テスト:25%)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1234, shuffle=False)

In [89]:
# 訓練データをパラメータ調整用の検証データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=1234, shuffle=False)

### TupleDatasetを作成

In [90]:
from chainer.datasets import tuple_dataset

train_var = tuple_dataset.TupleDataset(X_train.astype(np.float32), y_train.astype(np.float32))

valid_var = tuple_dataset.TupleDataset(X_valid.astype(np.float32), y_valid.astype(np.float32))

test_var = tuple_dataset.TupleDataset(X_test.astype(np.float32), y_test.astype(np.float32))

### Iteratorの作成

In [91]:
from chainer import iterators

# 4 * 24 * 7 = 672レコード(1週間)毎に、パラメータを更新する。
batchsize = 672

# train:イテレータの名前、batchsize:ミニバッチサイズ(時系列は意識するので、shufleしない)
train_iter = iterators.SerialIterator(train_var, batchsize, shuffle=False)

valid_iter = iterators.SerialIterator(valid_var, batchsize, repeat=False, shuffle=False)

test_iter = iterators.SerialIterator(test_var, batchsize, repeat=False, shuffle=True)

### ライブラリーのインポート

In [92]:
# chainerと必要なパッケージをインポート
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import Chain, Variable, datasets, optimizers

from chainer import report, training
from chainer.training import extensions
import chainer.cuda

### LSTMネットワークの作成

In [93]:
class LSTM(Chain):
    def __init__(self):
        # Optimizerが最適化対象のパラメータをもつ事を暗示する。
        super(LSTM, self).__init__()

        with self.init_scope():                
            self.l1 = L.Linear(672, 200)
            self.l2 = L.LSTM(200, 200)
            self.l3 = L.Linear(200, 28)

    def reset_state(self):
        self.l2.reset_state()  
        
        
    def __call__(self, x):
        
        # データを受け取った際のforward計算
        h = F.relu(self.l1(x))
        h = F.dropout(h, ratio=0.7)
        
        h = F.relu( self.l2(h))
#         h = F.dropout(h, ratio=0.6)
        
        h = F.relu( self.l3(h))
    
        # マイナスをゼロに変換         
        return h
    

In [94]:
class LSTMRegressor(Chain):
    def __init__(self, predictor):
        super(LSTMRegressor, self).__init__(predictor=predictor)

    def __call__(self, x, y):
        pred = self.predictor(x)

        sqrt_diff = (y - pred)**2
        count_diff = np.sum(y != pred.data)

        loss = F.sqrt(F.sum(sqrt_diff) / count_diff)

        #         loss = F.mean_squared_error(pred, y)

        #         mse = ((y - pred)**2)
        #         loss = F.sqrt(F.sum(mse) / x.shape[0])
        #         loss = F.sqrt(F.sum(mse) / x.shape[1] * x.shape[0])

        #         loss = F.mean_absolute_error(pred, y)

        #         mae = F.absolute_error(y, pred)
        #         loss = F.sum(mae) / x.shape[0]

        report({'loss': loss}, self)

        return loss

### モデルの作成

In [95]:
model = LSTMRegressor(LSTM())

### オプティマイザの設定

In [96]:
from chainer import optimizers

# 最適化手法をAdamに指定
optimizer = optimizers.Adam(alpha=0.01)

# モデルにオプティマイザーを組み込む
optimizer.setup(model)

### Updaterの作成

In [97]:
from chainer import training
from chainer.cuda import to_cpu

updater = training.StandardUpdater(train_iter, optimizer)

### Trainerの準備

In [98]:
trainer = training.Trainer(updater, (100, 'epoch'))

trainer.extend(extensions.Evaluator(valid_iter, model))
trainer.extend(extensions.dump_graph('main/loss'))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(), trigger=(10, 'epoch'))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], file_name='loss.png'))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'elapsed_time']))
# trainer.extend(extensions.ProgressBar())

### 実行

In [99]:
trainer.run()

epoch       main/loss   validation/main/loss  elapsed_time
1           6.36624     5.94346               16.5993       
2           5.2445      4.8061                32.1837       
3           4.31384     4.30661               48.2618       
4           3.97513     4.05223               64.8422       
5           3.84148     3.98857               80.5221       
6           3.77777     3.95833               96.1965       
7           3.72266     3.89862               117.871       
8           3.71201     3.88292               142.795       
9           3.70571     3.82547               159.861       
10          3.66687     3.82239               179.072       
11          3.65346     3.88461               198.79        
12          3.62473     3.8639                215.459       
13          3.62857     3.87685               231.981       
14          3.62388     3.79598               249.166       
15          3.6091      3.73598               266.883       
16          3.59507     3.

KeyboardInterrupt: 

## モデルの保存

In [ ]:
from chainer import serializers

serializers.save_npz('lstm.model', model)

# check if the model is saved.
%ls -la lstm.model

## モデルの読み込み

In [ ]:
# Create the infrence (evaluation) model as the previous model
infer_model = LSTMRegressor(LSTM())

# Load the saved parameters into the parameters of the new inference model to overwrite
serializers.load_npz('lstm.model', infer_model)

## 学習、評価にも使用していないデータで検証

In [ ]:
import pydot
from IPython.display import Image, display

def show_graph():
    graph = pydot.graph_from_dot_file('result/cg.dot') # load from .dot file
    graph[0].write_png('graph.png')
    display(Image('graph.png', width=600, height=600))

In [ ]:
def show_loss():
    display(Image(filename='result/loss.png'))

In [ ]:
test_evaluator = extensions.Evaluator(test_iter, model)
results = test_evaluator()
print('Test loss:', results['main/loss'])

In [ ]:
test_batch = test_iter.next()

In [ ]:
from chainer.dataset import concat_examples
x, t = concat_examples(test_batch)

y = model.predictor(x)

In [ ]:
x.shape

In [ ]:
a = []
for i in range(0, 672):
    a.append(((t[i] - np.round(y[i].data))**2).sum())

# a.sum()/672
np.sqrt(np.array(a).sum() / 672)

In [ ]:
(3 -2.5)**2 + (-0.5 - 0)** 2 + (7 - 8)** 2 

In [ ]:
x.shape

In [ ]:
# display(y[0], t[0])
display(y[16].data, np.round(y[16].data), t[16])

In [ ]:
np.sqrt( ((t[0] -y[0].data)**2).sum())

In [ ]:
(t[0] -y[0].data)**2

In [ ]:
((t[0] -y[0].data)**2).sum()

In [ ]:
np.sqrt(21.348974)

In [57]:
a = np.array([[1,2,3], [2,4,9]])
b = np.array([[2,4,9], [1,2,3]])

mae = F.absolute_error(a.astype(np.float32) , b.astype(np.float32))
print(mae)
F.sum(mae) / a.shape[0]


variable([[1. 2. 6.]
          [1. 2. 6.]])


variable(9.)

In [60]:
a = a.astype(np.float32)
b = b.astype(np.float32)

np.sum(a != b)

6